In [17]:
bronze_location = "abfss://bronze@prsynapselab.dfs.core.windows.net/"
write_mode="append"


In [18]:
%%spark
val write_mode="append"


In [19]:
observationSchema = spark.read.json(bronze_location+"historic_data/observation/year=2016/part-00000-680e113d-60c6-468f-ba37-ea5ffd426dfe.c000.json").schema
# observationSchema = spark.read.json(bronze_location+"incremental_data/observation/year=2021/month=1/day=1/part-00000-6aadddce-f3ce-40c6-a5d0-d5f36d8fef75.c000.json").schema
observation_df = spark.read.schema(observationSchema).json(bronze_location+"incremental_data/observation/*/*/*/*.json")

In [20]:
observation_df.printSchema()

In [21]:
display(observation_df)

In [22]:
observation_df_selected = observation_df.select("id","resourceType","status","issued","subject","encounter","effectiveDateTime","valueQuantity","valueString","code","component","category") \
                        .toDF(*("observation_id","resourceType","status","issued","patient","encounter","effectiveDateTime","valueQuantity","valueString","code","component","category"))

In [23]:
observation_df_selected.createOrReplaceTempView("observation_df_selected")

In [24]:
%%spark
import org.apache.spark.sql.types.{StructType,ArrayType}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions.explode_outer

def flattenDataFrame(df: DataFrame): DataFrame = {
  val fields = df.schema.fields
  val fieldNames = fields.map(x => x.name)

  for (i <- fields.indices) {
    val field = fields(i)
    val fieldType = field.dataType
    val fieldName = field.name
    fieldType match {
      case _: ArrayType =>
        val fieldNamesExcludingArray = fieldNames.filter(_ != fieldName)
        val fieldNamesAndExplode = fieldNamesExcludingArray ++ Array(
          s"explode_outer($fieldName) as $fieldName"
        )
        val explodedDf = df.selectExpr(fieldNamesAndExplode: _*)
        return flattenDataFrame(explodedDf)
      case structType: StructType =>
        val childFieldNames =
          structType.fieldNames.map(childname => fieldName + "." + childname)
        val newFieldNames = fieldNames.filter(_ != fieldName) ++ childFieldNames
        import org.apache.spark.sql.functions.col

        val renamedCols =
          newFieldNames.map { x =>
            col(x.toString).as(x.toString.replace(".", "_"))
          }

        val explodedDf = df.select(renamedCols: _*)
        return flattenDataFrame(explodedDf)
      case _ =>
    }
  }

  df
}

## Creating observation Main Table

In [25]:
observation_df_selected.printSchema()

In [26]:
%%spark
val observation_main_df = spark.sql("select * from observation_df_selected").drop("code","component","category");


In [27]:
%%spark
val observation_main_df_flattened = flattenDataFrame(observation_main_df)

In [28]:
%%spark
display(observation_main_df_flattened)

In [38]:
%%spark
val silver_location = "abfss://silver@prsynapselab.dfs.core.windows.net/"
val write_mode="append"

observation_main_df_flattened.coalesce(200).write.format("delta").option("path", silver_location+"observation_main").mode(write_mode).saveAsTable("fhir.observation_main_hash")


## Creating observation Code Table

In [30]:
%%spark
val observation_code_df = spark.sql("select observation_id, code from observation_df_selected")
val observation_code_df_flattened = flattenDataFrame(observation_code_df)
display(observation_code_df_flattened)

In [31]:
%%spark
val silver_location = "abfss://silver@prsynapselab.dfs.core.windows.net/"
val write_mode="append"

observation_code_df_flattened.coalesce(200).write.format("delta").option("path", silver_location+"observation_code").mode(write_mode).saveAsTable("fhir.observation_code_hash")

## Creating observation component Table

In [32]:
%%spark
val observation_component_df = spark.sql("select observation_id, component from observation_df_selected")
val observation_component_df_flattened = flattenDataFrame(observation_component_df)
display(observation_component_df_flattened)

In [33]:
%%spark
val silver_location = "abfss://silver@prsynapselab.dfs.core.windows.net/"
val write_mode="append"

observation_component_df_flattened.coalesce(200).write.format("delta").option("path", silver_location+"observation_component").mode(write_mode).saveAsTable("fhir.observation_component_hash")

## Creating observation category Table

In [34]:
%%spark
val observation_category_df = spark.sql("select observation_id, category from observation_df_selected")
val observation_category_df_flattened = flattenDataFrame(observation_category_df)
display(observation_category_df_flattened)

In [35]:
%%spark
val silver_location = "abfss://silver@prsynapselab.dfs.core.windows.net/"
val write_mode="append"

observation_category_df_flattened.coalesce(200).write.format("delta").option("path", silver_location+"observation_category").mode(write_mode).saveAsTable("fhir.observation_category_hash")

for some reason my main table failed to register in previous step.   
```
%%sql

CREATE TABLE fhir.observation_main_hash USING DELTA LOCATION 'abfss://silver@prsynapselab.dfs.core.windows.net/observation_main'
```